In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [31]:
import numpy as np
from numpy.typing import ArrayLike
import json
from traitlets import observe
from pathlib import Path
data = json.loads(Path("/Users/mpiekenbrock/Downloads/miserables.json").read_text())

node_ids = np.unique([n["id"] for n in data["nodes"]])
src_ids = np.array([e["source"] for e in data["links"]])
tgt_ids = np.array([e["target"] for e in data["links"]])

src_ids = np.searchsorted(node_ids, src_ids)
tgt_ids = np.searchsorted(node_ids, tgt_ids)
node_ids = np.searchsorted(node_ids, node_ids)

edgelist = np.c_[src_ids, tgt_ids]

import traitlets
import anywidget
class Widget(anywidget.AnyWidget):
    _esm = "pixiplex/static/widget.js"
    _css = "pixiplex/static/widget.css"

    scale = traitlets.Float(2.0).tag(sync=True)
    width = traitlets.Int(250).tag(sync=True)
    height = traitlets.Int(250).tag(sync=True)
    
    node_ids = traitlets.List(trait=traitlets.CInt).tag(sync=True)
    src_ids = traitlets.List(trait=traitlets.CInt).tag(sync=True)
    tgt_ids = traitlets.List(trait=traitlets.CInt).tag(sync=True)

    _x = traitlets.List(trait=traitlets.Float).tag(sync=True)
    _y = traitlets.List(trait=traitlets.Float).tag(sync=True)

    node_color = traitlets.List(trait=traitlets.Unicode).tag(sync=True)
    
    def __init__(self, node_ids: ArrayLike, edgelist: ArrayLike, width: int = 250, height: int = 250) -> None:
        super().__init__()
        self.width = width 
        self.height = height
        self.node_ids = list(node_ids)
        self.src_ids = list(edgelist[:,0])
        self.tgt_ids = list(edgelist[:,1])

    @property
    def x(self):
        self.send({ "type" : "msg:sync_node_coordinates"})
        return self._x

    @x.setter
    def x(self, value: ArrayLike):
        value = np.atleast_1d(value).astype(np.float32).ravel();
        assert len(value) == len(self.node_ids)
        self._x = value

    def center(self, fit_zoom: bool = False, x: float = None, y: float = None):
        msg = { "type" : "msg:center", "fit": fit_zoom }
        msg['x'] = x if x is not None else (self.width * self.scale / 2)
        msg['y'] = y if x is not None else (self.height * self.scale / 2)
        self.send(msg)

widget = Widget(node_ids, edgelist, width=750, height=450)
widget

Widget(height=450, node_ids=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,…

In [32]:
widget.center(True)

In [24]:
# widget.node_color = ["0xff0000"]*len(widget.node_ids)
widget.node_color = ["0xff0000" if i % 2 == 0 else "0x00ff00" for i in widget.node_ids]

In [26]:
widget.x

[401.25419103962145,
 223.9083080362283,
 300.5539044258302,
 17.01439921303288,
 586.7960682792517,
 618.4171469296709,
 251.50625061735965,
 743.9343613268796,
 175.2757856258349,
 285.78657238636583,
 671.9853255132901,
 673.5939548760202,
 569.1032458021788,
 470.250971029266,
 657.1126471411909,
 551.8977816793304,
 690.6870658189558,
 436.60341319971906,
 493.9310776535165,
 747.583300563252,
 372.0432051163273,
 722.2211594926338,
 466.88536361454993,
 384.84145211891564,
 26.45147737258918,
 588.7985491726118,
 552.2364846986205,
 610.903608662883,
 226.16494315414315,
 340.28902601754106,
 246.46593723190884,
 511.9879663022353,
 394.6524705243478,
 251.80025962096414,
 372.6813774100666,
 729.5824461064149,
 7.064791040054974,
 519.9134760932137,
 139.53117847796403,
 500.27781146313896,
 206.27370913711368,
 720.0770200747063,
 513.9765214825205,
 573.4462230385681,
 707.7235255527398,
 713.289440545866,
 359.06490442254824,
 184.54244234003647,
 360.16804415553173,
 25.2254

In [59]:
widget.send({ "type" : "msg:sync_node_coordinates"})
# widget._x

In [35]:

print(widget._x)
widget.send({ "type" : "msg:sync_node_coordinates"})
print(widget._x)

[]
[]


In [17]:
import asyncio
await asyncio.gather(widget.x())

/var/folders/0l/b3dbb2_d2bb4y3wbbfk0wt_80000gn/T/ipykernel_16472/4165577115.py:2: RuntimeWarning: coroutine 'Widget.x' was never awaited
  await asyncio.gather(widget.x())


TypeError: 'coroutine' object is not callable

In [6]:
wut = widget.send({ "type" : "msg:sync_node_coordinates"})
print(wut)

None


In [17]:
widget.send({ "type" : "msg:init_graph"})

In [60]:
widget.send({ "type" : "msg:init_force"})

In [61]:
widget.send({ "type" : "msg:center_graph"})

In [5]:
widget.send({ "type" : "msg:custom", "foo":"bar" })

In [ ]:
# %% Embed statically -- this works!
# import ipywidgets.embed
# ipywidgets.embed.embed_minimal_html("forcegraph_embed.html", views=[widget], drop_defaults=False)